In [ ]:
import numpy as np
import pandas as pd

import math
from keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, SimpleRNN, Conv1D, GlobalMaxPooling1D, Flatten, MaxPooling1D
from keras.layers.wrappers import Bidirectional

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error

from keras.preprocessing.sequence import TimeseriesGenerator

In [ ]:
# load the dataset from static csv
# df = pd.read_csv('data/PFG.csv', header=0, index_col=0, parse_dates=True, usecols=['Date','Adj Close'])
# df.head(2)

### Load Data using Pandas Datareader

In [ ]:
!pip install pandas-datareader

In [ ]:
!pip install yfinance --upgrade --no-cache-dir

In [ ]:
import pandas_datareader.data as pdr
import yfinance as yf

yf.pdr_override()
df = pdr.get_data_yahoo('PFG')#, start, end)

df = df[['Adj Close']]  
df.head()

In [ ]:
df.tail()

In [ ]:
dataset = df.values.astype('float32')
dataset

In [ ]:
dataset.shape

In [ ]:
split_pct = 0.8

split = int(split_pct*len(dataset))

train = dataset[:split]
valid = dataset[split:]

print(len(train))
print(len(valid))

train.shape, valid.shape

In [ ]:
# skip this cell if you don't want scaling

#scaler = MinMaxScaler(feature_range=(0, 1)) 
scaler = StandardScaler(with_mean=True, with_std=True)
train = scaler.fit_transform(train)
valid = scaler.transform(valid)

train.shape, valid.shape

#### Generate Timeseries Sequences

https://keras.io/api/preprocessing/timeseries/#timeseriesgenerator-class

In [ ]:
lags = 30

train_generator = TimeseriesGenerator(train, train, length=lags, batch_size=5)
valid_generator = TimeseriesGenerator(valid, valid, length=lags, batch_size=1)

In [ ]:
train_generator[0][1]

### Build Models

#### Simple RNN

In [ ]:
model = Sequential()
model.add(SimpleRNN(256, dropout=0.2, input_shape=(lags, 1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(train_generator, epochs=10, verbose=1, validation_data=valid_generator)

#### LSTM

In [ ]:
model = Sequential()
model.add(LSTM(100, input_shape=(lags, 1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(train_generator, epochs=10, verbose=1, validation_data=valid_generator)

#### Stacked LSTM

In [ ]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(lags, 1)))
model.add(LSTM(50, dropout=0.2))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(train_generator, epochs=10, verbose=1, validation_data=valid_generator)

#### Bidirectional LSTM

In [ ]:
model = Sequential()
model.add (Bidirectional(LSTM(100, input_shape=(lags, 1))))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(train_generator, epochs=10, verbose=1, validation_data=valid_generator)

#### Stacked Bidirectional LSTM

In [ ]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(lags, 1)))
model.add(Bidirectional(LSTM(50, dropout=0.2)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(train_generator, epochs=10, verbose=1, validation_data=valid_generator)

#### CNN-LSTM

In [ ]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(lags, 1)))
model.add(MaxPooling1D())
model.add(LSTM(100))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(train_generator, epochs=10, verbose=1, validation_data=valid_generator)

#### Bidirectional CNN-LSTM

In [ ]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(lags, 1)))
model.add(MaxPooling1D())
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(train_generator, epochs=10, verbose=1, validation_data=valid_generator)

### Evaluate the Model

In [ ]:
math.sqrt(2.6903)

In [ ]:
# skip this cell if you used scaling
y_valid = np.array([t[1][0] for t in valid_generator])
y_preds=model.predict(valid_generator)

y_valid.shape

In [ ]:
# skip this cell if you didn't scale
y_valid = scaler.inverse_transform(np.array([t[1][0] for t in valid_generator]))
y_preds=scaler.inverse_transform(model.predict(valid_generator))

y_preds.shape , y_valid.shape

In [ ]:
math.sqrt(mean_squared_error(y_valid.reshape(-1), y_preds.reshape(-1)))

In [ ]:
y_preds.reshape(-1)[0:3]

In [ ]:
y_valid.reshape(-1)[0:3]

In [ ]:
def ts_predict(data, lags, num_periods, model, scaler=None):
    if scaler is not None:
      preds = scaler.transform(data[-lags:])
    
      for _ in range(num_periods):
          preds = np.append(preds, model.predict(preds[-lags:].reshape((1, lags, 1)))[0][0])
          
      return scaler.inverse_transform(preds[lags:])
    else:
      preds = data[-lags:]
    
      for _ in range(num_periods):
          preds = np.append(preds, model.predict(preds[-lags:].reshape((1, lags, 1)))[0][0])
          
      return preds[lags:]

In [ ]:
dataset[-lags:]

In [ ]:
ts_predict(dataset, lags=lags, num_periods=5, model=model, scaler=scaler)

In [ ]:
ts_predict(dataset, lags=lags, num_periods=3, model=model, scaler=scaler)

In [ ]:
ts_predict(dataset, lags=lags, num_periods=5, model=model)